# Preprocessing Pipeline

Notebook version of `notebooks/preprocessing.py`.

Run all cells to regenerate:
- `data/processed/Training_Dataset_Preprocessed.csv`
- `data/processed/Test_Dataset_Preprocessed.csv`
- `data/processed/common_genes.txt`

In [1]:
"""
PREPROCESSING PIPELINE DOCUMENTATION
====================================

This script documents the complete preprocessing steps:
1. Load raw GEO series matrix files
2. Load GPL annotation files  
3. Map probe IDs to gene symbols
4. Log2 transformation
5. Handle missing values
6. Merge datasets
7. Assign labels

Original preprocessing by project author - documented here for transparency
"""

import pandas as pd
import numpy as np
from io import StringIO
import re
from pathlib import Path

# Define file paths
if "__file__" in globals():
    PROJECT_ROOT = Path(__file__).resolve().parents[1]
elif Path.cwd().name == "notebooks":
    PROJECT_ROOT = Path.cwd().parent
else:
    PROJECT_ROOT = Path.cwd()

RAW_DATA_PATH = str(PROJECT_ROOT / "data" / "raw") + "/"
ANNOTATION_PATH = str(PROJECT_ROOT / "data" / "annotation") + "/"
OUTPUT_PATH = str(PROJECT_ROOT / "data" / "processed") + "/"

Path(OUTPUT_PATH).mkdir(parents=True, exist_ok=True)

# Dataset to GPL mapping
# Training datasets (5)
TRAINING_DATASETS = {
    'GSE24124': ('GPL887-20438.txt', 'GPL887'),
    'GSE32641': ('GPL887-20438.txt', 'GPL887'),
    'GSE36295': ('GPL6244-17930.txt', 'GPL6244'),
    'GSE42568': ('GPL570-55999.txt', 'GPL570'),
    'GSE53752': ('GPL7264-9589.txt', 'GPL7264')
}

# Test datasets (2)
TEST_DATASETS = {
    'GSE70947': ('GPL13607-20416.txt', 'GPL13607'),
    'GSE109169': ('GPL5175-3188.txt', 'GPL5175')
}

# All datasets
ALL_DATASETS = {**TRAINING_DATASETS, **TEST_DATASETS}

def parse_gpl_file(gpl_file, platform_type):
    """
    Parse GPL annotation file based on platform type
    """
    print(f"  Parsing {gpl_file}...")
    
    if platform_type == 'GPL887':  # Agilent
        gpl_df = pd.read_csv(ANNOTATION_PATH + gpl_file, sep="\t", comment="#", low_memory=False)
        probe_map = gpl_df[['ID', 'GENE_SYMBOL']].dropna()
        
    elif platform_type == 'GPL6244':  # Affy Gene 1.0 ST
        gpl_df = pd.read_csv(ANNOTATION_PATH + gpl_file, sep="\t", comment="#", low_memory=False)
        def extract_gene(text):
            if pd.isna(text): return None
            parts = [p.strip() for p in str(text).split("//")]
            return parts[1] if len(parts) >= 2 and parts[1] != '---' else None
        gpl_df['GENE_SYMBOL'] = gpl_df['gene_assignment'].apply(extract_gene)
        probe_map = gpl_df[['ID', 'GENE_SYMBOL']].dropna()
        
    elif platform_type == 'GPL570':  # Affy U133 Plus 2.0
        probe_ids, gene_symbols = [], []
        with open(ANNOTATION_PATH + gpl_file, 'r', encoding='utf-8') as f:
            for line in f:
                if line.startswith(("ID", "!", "#")):
                    continue
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                probe_id = fields[0]
                line_text = ' '.join(fields)
                match = re.search(r"\s([A-Z0-9\-]{2,})\s+\d{1,6}", line_text)
                if match:
                    gene = match.group(1)
                    if gene.upper() not in ["HOMO", "NA"]:
                        probe_ids.append(probe_id)
                        gene_symbols.append(gene)
        probe_map = pd.DataFrame({"ID": probe_ids, "GENE_SYMBOL": gene_symbols})
        
    elif platform_type == 'GPL7264':  # Affy Exon 1.0 ST
        probe_ids, gene_symbols = [], []
        with open(ANNOTATION_PATH + gpl_file, 'r', encoding='utf-8') as f:
            for line in f:
                if line.startswith(("ID", "!", "#")):
                    continue
                fields = line.strip().split('\t')
                if len(fields) < 7:
                    continue
                probe_id = fields[0]
                gene = fields[6] if len(fields) > 6 else None
                if gene and gene != '---':
                    probe_ids.append(probe_id)
                    gene_symbols.append(gene)
        probe_map = pd.DataFrame({"ID": probe_ids, "GENE_SYMBOL": gene_symbols})
        
    elif platform_type == 'GPL13607':  # Affy Gene 2.0 ST
        gpl_df = pd.read_csv(ANNOTATION_PATH + gpl_file, sep="\t", comment="#", low_memory=False)
        if 'GeneName' in gpl_df.columns:
            probe_map = gpl_df[['ID', 'GeneName']].rename(columns={'GeneName': 'GENE_SYMBOL'}).dropna()
        else:
            probe_map = gpl_df[['ID', 'GENE_SYMBOL']].dropna()
            
    elif platform_type == 'GPL5175':  # Affy HuEx-1_0-st
        gpl_df = pd.read_csv(ANNOTATION_PATH + gpl_file, sep="\t", comment="#", low_memory=False)
        def extract_first_gene(text):
            if pd.isna(text):
                return None
            match = re.search(r'//\s*([A-Za-z0-9\-\.]+)\s*//', str(text))
            return match.group(1) if match else None
        gpl_df['GENE_SYMBOL'] = gpl_df['gene_assignment'].apply(extract_first_gene)
        probe_map = gpl_df[['ID', 'GENE_SYMBOL']].dropna()
    
    else:
        raise ValueError(f"Unknown platform: {platform_type}")
    
    probe_map = probe_map.copy()
    probe_map["ID"] = probe_map["ID"].astype(str).str.strip()
    probe_map["GENE_SYMBOL"] = probe_map["GENE_SYMBOL"].astype(str).str.strip()
    probe_map = probe_map[(probe_map["GENE_SYMBOL"] != "") & (probe_map["GENE_SYMBOL"] != "---")]

    print(f"    Extracted {len(probe_map)} probe-to-gene mappings")
    return probe_map


def load_series_matrix(series_file):
    """
    Load GEO series matrix file
    """
    print(f"  Loading {series_file}...")
    
    with open(RAW_DATA_PATH + series_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    # Extract sample titles for labels
    sample_title_line = next((l for l in lines if l.startswith("!Sample_title")), None)
    if sample_title_line:
        sample_titles = [title.lower().strip().replace('"', '') 
                        for title in sample_title_line.strip().split("\t")[1:]]
    else:
        sample_titles = []
    
    # Extract GSM IDs
    sample_id_line = next((l for l in lines if l.startswith("!Sample_geo_accession")), None)
    if sample_id_line:
        gsm_ids = [s.strip().replace('"', '') 
                  for s in sample_id_line.strip().split("\t")[1:]]
    else:
        gsm_ids = [f"Sample_{i}" for i in range(len(sample_titles))]
    
    # Read expression data
    data_lines = [line for line in lines if not line.startswith("!")]
    expr_df = pd.read_csv(StringIO("".join(data_lines)), sep="\t", low_memory=False)
    expr_df = expr_df.rename(columns={expr_df.columns[0]: "ProbeID"})
    expr_df['ProbeID'] = expr_df['ProbeID'].astype(str).str.strip()
    expr_df = expr_df.set_index("ProbeID")
    expr_df = expr_df.apply(pd.to_numeric, errors='coerce')
    
    # Remove probes with >50% missing values
    expr_df = expr_df.dropna(thresh=int(expr_df.shape[1] * 0.5))
    
    print(f"    Loaded {expr_df.shape[0]} probes x {expr_df.shape[1]} samples")
    return expr_df, sample_titles, gsm_ids


def assign_labels(sample_titles):
    """
    Assign binary labels based on sample titles
    0 = Normal, 1 = Tumor
    """
    labels = []
    disease_types = []
    valid_indices = []
    
    for i, title in enumerate(sample_titles):
        title_lower = title.lower()
        
        # Check for normal/control
        if any(term in title_lower for term in ['normal', 'control', 'nontumor', 'healthy', 'adjacent']):
            labels.append(0)
            disease_types.append('normal')
            valid_indices.append(i)
        # Check for tumor/cancer
        elif any(term in title_lower for term in ['tumor', 'cancer', 'carcinoma', 'idc', 'malignant']):
            labels.append(1)
            disease_types.append('tumor')
            valid_indices.append(i)
    
    return labels, disease_types, valid_indices


def preprocess_dataset(dataset_name, gpl_file, platform_type):
    """
    Preprocess a single dataset
    """
    print(f"\nProcessing {dataset_name}...")
    
    # 1. Load GPL annotation
    probe_map = parse_gpl_file(gpl_file, platform_type)
    
    # 2. Load series matrix
    series_file = f"{dataset_name}_series_matrix.txt"
    expr_df, sample_titles, gsm_ids = load_series_matrix(series_file)
    
    # 3. Map probes to genes
    common_probes = expr_df.index.intersection(probe_map['ID'].astype(str).values)
    expr_subset = expr_df.loc[common_probes]
    probe_to_gene = dict(zip(probe_map['ID'], probe_map['GENE_SYMBOL']))
    expr_subset['GeneSymbol'] = expr_subset.index.map(probe_to_gene)
    expr_subset = expr_subset.dropna(subset=['GeneSymbol'])
    gene_expr = expr_subset.groupby('GeneSymbol').mean()
    
    print(f"    Mapped to {len(gene_expr)} unique genes")
    
    # 4. Transpose so samples are rows
    gene_expr = gene_expr.T
    
    # 5. Assign labels
    labels, disease_types, valid_indices = assign_labels(sample_titles)
    gene_expr = gene_expr.iloc[valid_indices]
    
    # 6. Add metadata
    gene_expr['GSM_ID'] = [gsm_ids[i] for i in valid_indices]
    gene_expr['DiseaseType'] = disease_types
    gene_expr['label'] = labels
    gene_expr['Dataset'] = dataset_name
    
    # 7. Log2 transform if needed
    gene_cols = [c for c in gene_expr.columns if c not in ['GSM_ID', 'DiseaseType', 'label', 'Dataset']]
    if gene_expr[gene_cols].max().max() > 100:
        print(f"    Applying log2 transformation...")
        gene_expr[gene_cols] = np.log2(gene_expr[gene_cols] + 1)
    
    print(f"    Final: {gene_expr.shape[0]} samples, {len(gene_cols)} genes")
    print(f"    Labels: Tumor={sum(labels)}, Normal={len(labels)-sum(labels)}")
    
    return gene_expr


def merge_datasets(datasets):
    """
    Merge all datasets by common genes
    """
    print("\n" + "="*70)
    print("MERGING DATASETS")
    print("="*70)
    
    # Find common genes
    meta_cols = ['GSM_ID', 'DiseaseType', 'label', 'Dataset']
    gene_sets = []
    
    for name, df in datasets.items():
        genes = [c for c in df.columns if c not in meta_cols]
        gene_sets.append(set(genes))
        print(f"  {name}: {len(genes)} genes")
    
    # Intersection
    common_genes = gene_sets[0]
    for gene_set in gene_sets[1:]:
        common_genes = common_genes.intersection(gene_set)
    
    common_genes = sorted(list(common_genes))
    print(f"\n  Common genes: {len(common_genes)}")
    
    # Merge
    merged = []
    for name, df in datasets.items():
        cols = meta_cols + common_genes
        merged.append(df[cols])
    
    merged_df = pd.concat(merged, ignore_index=True)
    
    print(f"\n  Merged dataset: {merged_df.shape}")
    print(f"  Samples: {len(merged_df)}")
    print(f"  Labels: {merged_df['label'].value_counts().to_dict()}")
    print(f"  Datasets: {merged_df['Dataset'].value_counts().to_dict()}")
    
    return merged_df, common_genes


def main():
    """
    Main preprocessing pipeline for all 7 datasets
    """
    print("="*70)
    print("PREPROCESSING PIPELINE - ALL 7 DATASETS")
    print("="*70)
    print("\nSteps:")
    print("1. Load GPL annotation files (probe -> gene mapping)")
    print("2. Load GEO series matrix files (raw expression)")
    print("3. Map probe IDs to gene symbols")
    print("4. Log2 transformation")
    print("5. Handle missing values")
    print("6. Assign labels (0=Normal, 1=Tumor)")
    print("7. Merge datasets by common genes")
    print("="*70)
    
    print("\n" + "="*70)
    print("PROCESSING TRAINING DATASETS (5 datasets)")
    print("="*70)
    
    # Process training datasets
    train_datasets = {}
    for dataset, (gpl_file, platform) in TRAINING_DATASETS.items():
        try:
            print(f"\n[{len(train_datasets)+1}/5] Processing training dataset: {dataset}")
            df = preprocess_dataset(dataset, gpl_file, platform)
            train_datasets[dataset] = df
        except Exception as e:
            print(f"  ERROR: {e}")
            continue
    
    print("\n" + "="*70)
    print("PROCESSING TEST DATASETS (2 datasets)")
    print("="*70)
    
    # Process test datasets
    test_datasets = {}
    for dataset, (gpl_file, platform) in TEST_DATASETS.items():
        try:
            print(f"\n[{len(test_datasets)+1}/2] Processing test dataset: {dataset}")
            df = preprocess_dataset(dataset, gpl_file, platform)
            test_datasets[dataset] = df
        except Exception as e:
            print(f"  ERROR: {e}")
            continue
    
    # Merge training datasets
    print("\n" + "="*70)
    print("MERGING TRAINING DATASETS")
    print("="*70)
    
    if train_datasets:
        train_merged, common_genes = merge_datasets(train_datasets)
        
        # Save training data
        train_output = OUTPUT_PATH + "Training_Dataset_Preprocessed.csv"
        train_merged.set_index('GSM_ID').to_csv(train_output)
        
        print(f"\n[SAVED] Training data: {train_output}")
        print(f"  Shape: {train_merged.shape}")
        print(f"  Samples: {len(train_merged)}")
    
    # Merge test datasets
    print("\n" + "="*70)
    print("MERGING TEST DATASETS")
    print("="*70)
    
    if test_datasets:
        # Find common genes with training data
        test_merged, _ = merge_datasets(test_datasets)
        
        # Keep only genes present in training data
        meta_cols = ['GSM_ID', 'DiseaseType', 'label', 'Dataset']
        train_genes = [c for c in train_merged.columns if c not in meta_cols]
        available_genes = [g for g in train_genes if g in test_merged.columns]
        
        test_merged_filtered = test_merged[meta_cols + available_genes]
        
        # Save test data
        test_output = OUTPUT_PATH + "Test_Dataset_Preprocessed.csv"
        test_merged_filtered.set_index('GSM_ID').to_csv(test_output)
        
        print(f"\n[SAVED] Test data: {test_output}")
        print(f"  Shape: {test_merged_filtered.shape}")
        print(f"  Samples: {len(test_merged_filtered)}")
        print(f"  Common genes with training: {len(available_genes)}")
    
    # Save gene list
    with open(OUTPUT_PATH + "common_genes.txt", "w") as f:
        for gene in common_genes:
            f.write(gene + "\n")
    
    print("\n" + "="*70)
    print("PREPROCESSING COMPLETE!")
    print("="*70)
    print("\nAll 7 datasets processed:")
    print(f"  Training: {len(train_datasets)} datasets, {len(train_merged) if train_datasets else 0} samples")
    print(f"  Test: {len(test_datasets)} datasets, {len(test_merged_filtered) if test_datasets else 0} samples")
    print(f"  Common genes: {len(common_genes)}")
    print("\nFiles generated:")
    print("  - Training_Dataset_Preprocessed.csv")
    print("  - Test_Dataset_Preprocessed.csv")
    print("  - common_genes.txt")
    print("="*70)
    
    return train_datasets, test_datasets


if __name__ == "__main__":
    train_datasets, test_datasets = main()


PREPROCESSING PIPELINE - ALL 7 DATASETS

Steps:
1. Load GPL annotation files (probe -> gene mapping)
2. Load GEO series matrix files (raw expression)
3. Map probe IDs to gene symbols
4. Log2 transformation
5. Handle missing values
6. Assign labels (0=Normal, 1=Tumor)
7. Merge datasets by common genes

PROCESSING TRAINING DATASETS (5 datasets)

[1/5] Processing training dataset: GSE24124

Processing GSE24124...
  Parsing GPL887-20438.txt...
    Extracted 19168 probe-to-gene mappings
  Loading GSE24124_series_matrix.txt...
    Loaded 22153 probes x 119 samples
    Mapped to 16546 unique genes
    Final: 119 samples, 16546 genes
    Labels: Tumor=99, Normal=20

[2/5] Processing training dataset: GSE32641

Processing GSE32641...
  Parsing GPL887-20438.txt...
    Extracted 19168 probe-to-gene mappings
  Loading GSE32641_series_matrix.txt...
    Loaded 22153 probes x 102 samples
    Mapped to 16546 unique genes
    Final: 102 samples, 16546 genes
    Labels: Tumor=95, Normal=7

[3/5] Process